## Deploy an nlproc.tools Interface to Prolific
In this tutorial, we will walk through a deployment to the [**Prolific**](https://app.prolific.co/) crowdsourcing platform. We will create a draft study and test it out. We assume you have created a Prolific account.

### Prepare Data for Crowdsourcing
In your template, make sure you have added crowdsourcing options:
```yml
crowdsource: "prolific"
prolific_completion_code: XXXXXXXX
disable:
- upload
- download
```
For this tutorial, we will be using the [**demo crowdsourcing template**](https://nlproc.tools/?t=demo_crowdsource), but any nlproc.tools template will work.

In [1]:
!pip install -q nlproc_tools

In [2]:
from nlproc_tools import load_interface

In [3]:
# Let's begin by downloading the demo typology and data
!curl -so crowdsource.yml https://nlproc.tools/templates/demo/crowdsource.yml
!curl -so crowdsource.json https://nlproc.tools/data/demo/crowdsource.json

In [4]:
# Now we can load and customize our interface template YML file we created on nlproc.tools
Interface = load_interface("crowdsource.yml")
print(f"Loaded interface: {Interface.template_label}")

Loaded interface: Crowdsourcing Demo


In [5]:
# Studies use a unique completion code. We begin by pasting the code into
# the interface template.
completion_code = "YOUR_COMPLETION_CODE"
Interface.prolific_completion_code = completion_code
print(Interface)

Demo(
  crowdsource: prolific, 
  prolific_completion_code: YOUR_COMPLETION_CODE, 
  disable: ['upload', 'download'], 
  instructions: ## Crowdsourcing Demo
  You can add instructions here. Click to the end and press "Submit" to be redirected back to Prolific
  , 
  prepend_instructions: True, 
  template_name: demo, 
  template_label: Crowdsourcing Demo, 
  edits: [{'name': 'edit_name', 'label': 'Edit Name', 'color': 'red', 'icon': 'fa-magnifying-glass', 'enable_input': True, 'enable_output': True}], 
  entry_class: <class 'nlproc_tools.names.DemoEntry'>, 
  edit_classes: {'edit_name': <class 'nlproc_tools.names.EditNameEdit'>}, 
  annotation_classes: {}
)


In [6]:
# Let's use the get_entry_class() to create an example for annotation
DemoEntry = Interface.get_entry_class()

entry = DemoEntry(
    source="Gather ye annotations on the prolific tome, that its wisdom and verity may be preserved for generations hence, like jewels adorning the literary crown.",
    target="Gather your annotations on the prolific book so that its wisdom and truth can be preserved for future generations, like precious jewels decorating the literary heritage."
)

print(entry)

DemoEntry(
  source: Gather ye annotations on the prolific tome, that its wisdom and verity may be preserved for generations hence, like jewels adorning the literary crown., 
  target: Gather your annotations on the prolific book so that its wisdom and truth can be preserved for future generations, like precious jewels decorating the literary heritage.
)


In [7]:
# Save our modified interface and custom data
# Interface.export_interface("crowdsource.yml")
# Interface.export_data([entry], "crowdsource.json")

In [8]:
# Create path files (for next section)
template_path, data_path = None, None

### [Optional] Deploy Your Interface using GitHub
Since Prolific needs to link to a hosted interface, we will deploy our interface publically using GitHub for hosting. You can skip this step by going to the [**Preview Interface**](#preview-interface) section.

For this section you will need to create a GitHub access token: [**github.com/settings/tokens**](https://github.com/settings/tokens). Make sure to give permissions for *Administration* and *Contents* so we can push to a repository.

In [9]:
!pip install -q gitpython PyGithub

In [11]:
import git, os, shutil
from github import Github

local_template_path = "crowdsource.yml"
local_data_path = "crowdsource.json"
repository_name = "prolific-deployment-demo"

# Get GitHub credentials
if os.path.exists('.git_credentials'):
    # This will read the API token from a local file
    github_username, github_token = open('.git_credentials', "r").read().split('\n')
else:
    github_username = input("Please enter your GitHub username: ")
    github_token = getpass("Please enter your GitHub access token (register a token at github.com/settings/tokens): ")

# Log into GitHub
g = Github(github_token)
user = g.get_user()

# Create a new repository for our interface
try:
    remote_repo = g.get_repo(f"{github_username}/{repository_name}").clone_url
    print(f"Found existing repository: {remote_repo}")
except Exception as e:
    remote_repo = user.create_repo(repository_name).clone_url
    print(f"Created new repository: {remote_repo}")

if not os.path.exists(repository_name):
    git.Repo.clone_from(remote_repo, repository_name)

repo = git.Repo(repository_name)

# Upload our template and data files
for file_path in [local_template_path, local_data_path]:
    template_repo_path = os.path.join(os.getcwd(), repository_name, file_path)
    shutil.copy(file_path, template_repo_path)
    repo.index.add([template_repo_path])
repo.index.commit("Add a new interface template and data")

origin = repo.remote("origin")
origin.push()
print(f"Pushed changes to {remote_repo}")

Created new repository: https://github.com/davidheineman/prolific-deployment-demo.git
Pushed changes to https://github.com/davidheineman/prolific-deployment-demo.git


In [12]:
template_path = f"{github_username}/{repository_name}/master/{local_template_path}"
data_path = f"{github_username}/{repository_name}/master/{local_data_path}"

print(f"Hosted template: {template_path}")
print(f"Hosted data: {data_path}")

Hosted template: davidheineman/prolific-deployment-demo/master/crowdsource.yml
Hosted data: davidheineman/prolific-deployment-demo/master/crowdsource.json


<a id="preview-interface"></a>

### Preview Interface
Now we can use nlproc.tools to view our hosted interface!

In [13]:
# If we skipped the above step, we will use the demo template and data
if not template_path and not data_path:
    template_path = "davidheineman/nlproc.tools/main/public/templates/demo/crowdsource.yml"
    data_path = "davidheineman/nlproc.tools/main/public/data/demo/crowdsource.json"

study_url = f"https://nlproc.tools/?gh={template_path}"
study_url = study_url + "&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}"

print(f"View your hosted template here:\t\t  {study_url}")

study_url = f"{study_url}&d={data_path}"

print(f"Your study (with data) will be linked to: {study_url}")

View your hosted template here:		  https://nlproc.tools/?gh=davidheineman/prolific-deployment-demo/master/crowdsource.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}
Your study (with data) will be linked to: https://nlproc.tools/?gh=davidheineman/prolific-deployment-demo/master/crowdsource.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}&d=davidheineman/prolific-deployment-demo/master/crowdsource.json


If everything works, you should see this:
<div align="center">
    <img src="../public/img/prolific-gh-hosted.png" width="100%" style="max-width: 400px" />
</div>

### Deploy to Prolific
We will deploy entirely through Python, which will help manage data in large-scale annotation scenarios. For details on the Prolific API, see [**docs.prolific.co/docs/api-docs/public**](https://docs.prolific.co/docs/api-docs/public).

In [14]:
# We will use Dallinger, a popular annotation service management tool
# Dallinger also has integrations with other services, such as MTurk
!pip install -q dallinger

In [16]:
from dallinger.prolific import ProlificService
import os

In [17]:
# To get a developer token, visit your prolific workspace -> settings -> API tokens
YOUR_PROLIFIC_API_TOKEN = "YOUR_API_TOKEN" 

# This will read the API token from a local file
if os.path.exists('.prolific_token'):
    YOUR_PROLIFIC_API_TOKEN = open('.prolific_token', "r").read()

prolific = ProlificService(
    api_token=YOUR_PROLIFIC_API_TOKEN,
    api_version="v1",
    referer_header=None
)

In [18]:
# Prolific annotations are managed by workspaces and grouped by projects.
# We will start by finding a workspace to use for annotation.
response = prolific._req(
    method="GET",
    endpoint=f"/workspaces/",
)
workspace_id = response['results'][0]['id']
print(f"Found {response['meta']['count']} workspace(s), using {response['results'][0]['title']} with id {workspace_id}")

2023-07-29 18:39:46,811 - WARNING - Prolific API request: {"URL": "https://api.prolific.co/api/v1/workspaces/", "method": "GET", "args": {}}


Found 1 workspace(s), using My Workspace with id 64c55f10b177aed40ec785a4


In [19]:
# Create a new project for our demo annoations.
response = prolific._req(
    method="POST",
    endpoint=f"/workspaces/{workspace_id}/projects/",
    json={
        "title": "My Demo nlproc.tools Project"
    },
)
project_id = response['id']
print(f"Created new project with id {project_id}")

2023-07-29 18:39:49,718 - WARNING - Prolific API request: {"URL": "https://api.prolific.co/api/v1/workspaces/64c55f10b177aed40ec785a4/projects/", "method": "POST", "args": {"json": {"title": "My Demo nlproc.tools Project"}}}


Created new project with id 64c5bf65d65dd689969fc570


In [22]:
# Create a new draft study. A seperate study should be created for every annotation task.

# For more information about survey options, refer to:
#   https://docs.prolific.co/docs/api-docs/public/#tag/Studies/paths/~1api~1v1~1studies~1/post
response = prolific.draft_study(
    # Information about the study
    name="My Demo Annotation Task",
    internal_name="demo-annotation-task",
    description="""In this task, you will use nlproc.tools to perform fine-grained evaluation!""",
    # project=project_id,

    # Parameters for your data collection
    estimated_completion_time=3,
    maximum_allowed_time=10,
    reward=8,
    total_available_places=1,
    eligibility_requirements=[],

    # Configuration for our nlproc.tools setup
    external_study_url=study_url,
    completion_code=completion_code,
    completion_option="url",
    prolific_id_option="url_parameters",
    device_compatibility=["desktop"],
    peripheral_requirements=[],
)
study_id = response['id']

print(f"Created draft study with name {response['name']} and ID {study_id}.")

2023-07-29 18:40:36,855 - WARNING - Prolific API request: {"URL": "https://api.prolific.co/api/v1/studies/", "method": "POST", "args": {"json": {"name": "My Demo Annotation Task", "internal_name": "demo-annotation-task", "description": "In this task, you will use nlproc.tools to perform fine-grained evaluation!", "external_study_url": "https://nlproc.tools/?gh=davidheineman/prolific-deployment-demo/master/crowdsource.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}&d=davidheineman/prolific-deployment-demo/master/crowdsource.json", "prolific_id_option": "url_parameters", "completion_code": "YOUR_COMPLETION_CODE", "completion_option": "url", "total_available_places": 1, "estimated_completion_time": 3, "maximum_allowed_time": 10, "reward": 4, "eligibility_requirements": [], "status": "UNPUBLISHED", "device_compatibility": ["desktop"], "peripheral_requirements": []}}}


Created draft study with name My Demo Annotation Task and ID 64c5bf94cf229332e5024cf6.


### View your study
Your draft study is now public! To view, visit your Prolific dashboard at [**app.prolific.co/researcher/workspaces**](https://app.prolific.co/researcher/workspaces).

<div align="center">
    <img src="../public/img/prolific.png" width="100%" style="max-width: 800px" />
</div>

To preview your study, scroll to the bottom and click on "Preview":

<div align="center">
    <img src="../public/img/prolific-study.png" width="100%" style="max-width: 800px" />
</div>

If we are successful, you will see your deployed interface!

<div align="center">
    <img src="../public/img/prolific-deployed.png" width="100%" style="max-width: 500px" />
</div>

In [23]:
print(f"View your study at: https://app.prolific.co/researcher/workspaces/studies/{study_id}")

View your study at: https://app.prolific.co/researcher/workspaces/studies/64c5bf94cf229332e5024cf6


### Manage Existing Studies
Now we have a working study, let's look at some more utility commands for managing the study

In [24]:
# We can add more slots to an existing study
response = prolific.add_participants_to_study(
    study_id=study_id,
    number_to_add=5
)

print(f"The study is now set to be open to {response['total_available_places']} participants.")

2023-07-29 18:41:16,894 - WARNING - Prolific API request: {"URL": "https://api.prolific.co/api/v1/studies/64c5bf94cf229332e5024cf6/", "method": "GET", "args": {}}
2023-07-29 18:41:17,329 - WARNING - Prolific API request: {"URL": "https://api.prolific.co/api/v1/studies/64c5bf94cf229332e5024cf6/", "method": "PATCH", "args": {"json": {"total_available_places": 6}}}


The study is now set to be open to 6 participants.


In [25]:
# When you are ready, this will make the study publically available to participants
# prolific.publish_study(study_id)

In [26]:
# This gets all the studies available account-wide
prolific.get_hits()

2023-07-29 18:41:23,195 - WARNING - Prolific API request: {"URL": "https://api.prolific.co/api/v1/studies/", "method": "GET", "args": {}}


[{'id': '64c5bf94cf229332e5024cf6',
  'title': 'My Demo Annotation Task',
  'annotation': 'demo-annotation-task',
  'status': 'UNPUBLISHED',
  'created': datetime.datetime(2023, 7, 30, 1, 40, 36, 945000, tzinfo=tzutc()),
  'expiration': '',
  'description': ''}]

### Cleanup
This will delete the resources created in this demo.

In [27]:
# Cleanup our resources
response = prolific.delete_study(study_id)
if response: print(f"Successfully deleted study: {study_id}!")

response = prolific._req(
    method="PATCH",
    endpoint=f"/projects/{project_id}/",
    json={},
)
if response: print(f"Successfully deleted project {project_id}!")

if 'g' in locals(): 
    g.get_repo(f"{github_username}/{repository_name}").delete()
    print(f"Successfully deleted GitHub repository {repository_name}!")

2023-07-29 18:41:26,137 - WARNING - Prolific API request: {"URL": "https://api.prolific.co/api/v1/studies/64c5bf94cf229332e5024cf6", "method": "DELETE", "args": {}}
2023-07-29 18:41:26,669 - WARNING - Prolific API request: {"URL": "https://api.prolific.co/api/v1/projects/64c5bf65d65dd689969fc570/", "method": "PATCH", "args": {"json": {}}}


Successfully deleted study: 64c5bf94cf229332e5024cf6!
Successfully deleted project 64c5bf65d65dd689969fc570!
Successfully deleted GitHub repository prolific-deployment-demo!


### Advanced features
Refer to the [**Dallinger library**](https://github.com/Dallinger/Dallinger/blob/master/dallinger/prolific.py) for details about the `ProlificService` class. Here are a few other useful commands:

- `pay_session_bonus()` - Pay a worker a bonus
- `approve_participant_session()` / `get_participant_session()` - Get / approve an annotation

Additionally, you can create your own API calls to Prolific using the format:
```python
response = prolific._req(
    method="XXX",               # The HTTP method, e.g., GET, POST
    endpoint=f"/XXX/YYY/",      # The API endpoint
    json={},                    # The request body in JSON format
)
```
For a full list of API calls and options, please refer to [**docs.prolific.co/docs/api-docs/public**](https://docs.prolific.co/docs/api-docs/public).